In [1]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from mlflow.models import infer_signature
import warnings

# suppress the MLflow model config warning if present
warnings.filterwarnings('ignore', message='Failed to log model config as params')

mlflow.set_experiment('model_search_guide')

# model configurations
model_configs = [
    {"model_type": "RandomForest", "n_estimators":100, "max_depth":10},
    {"model_type": "RandomForest","n_estimators":200, "max_depth":20},
    {"model_type": "LogisticRegression","C":1.0, "solver":"lbfgs"},
    {"model_type": "LogisticRegression","C":0.1, "solver":"saga"},
    {"model_type": "SVM", "kernel":"rbf", "C":1.0},
    {"model_type": "SVM","kernel":"linear","C":0.5}
    ]

# performance metrics
accuracy_scores = [0.92, 0.94, 0.88, 0.86, 0.90, 0.87]
precision_scores = [0.91, 0.93, 0.87, 0.85, 0.89, 0.86]
recall_scores = [0.93, 0.95, 0.89, 0.87, 0.91, 0.88]
f1_scores = [0.92, 0.94, 0.88, 0.86, 0.90, 0.87]

# model metadata 
versions = ["v1.0", "v1.1", "v1.0", "v2.0", "v1.0", "v1.1"]
environments = [
    'production','staging','production','development','staging','production'
]
frameworks = ["sklearn", "sklearn", "sklearn", "sklearn", "sklearn", "sklearn"]

# create dummy training data
X_train = np.random.rand(100,10)
y_train = np.random.randint(0, 2, 100)

# create input example for model signature
input_example = pd.DataFrame(X_train[:5], columns=[f'feature_{i}' for i in range(10)])

for i, config in enumerate(model_configs):
    with mlflow.start_run():
        # create and train model on type
        if config['model_type'] == 'RandomForest':
            model = RandomForestClassifier(
                n_estimators=config['n_estimators'],
                max_depth=config['max_depth']
            )

            mlflow.log_param('n_estimators', config['n_estimators'])
            mlflow.log_param('max_depth', config['max_depth'])

        elif config['model_type'] == 'LogisticRegression':
            model = LogisticRegression(
                C=config['C'],
                solver=config['solver']
            )

            mlflow.log_param('C', config['C'])
            mlflow.log_param('solver', config['solver'])

        else:
            model = SVC(
                kernel=config['kernel'],
                C=config['C'],
                random_state=42,
                probability=True
            )

            mlflow.log_param('kernel',config['kernel'])
            mlflow.log_param('C', config['C'])

        # log common parameters
        mlflow.log_param("model_type", config['model_type'])

        # fit model
        model.fit(X_train, y_train)

        # get predictions for signature
        predictions = model.predict(X_train[:5])

        # create model signature
        signature = infer_signature(X_train[:5], predictions)

        # log metrics
        mlflow.log_metric("accuracy", accuracy_scores[i])
        mlflow.log_metric('precision', precision_scores[i])
        mlflow.log_metric('recall', recall_scores[i])
        mlflow.log_metric('f1_score', f1_scores[i])

        # set tags
        mlflow.set_tag("version", versions[i])
        mlflow.set_tag("environment", environments[i])
        mlflow.set_tag("framework", frameworks[i])

        # log the model signature and input example
        model_name = f'{config['model_type']}_model_{i}'
        mlflow.sklearn.log_model(
            model, 
            name=model_name,
            signature=signature,
            input_example=input_example,
            registered_model_name=f'SearchGuide{config['model_type']}'
        )

c:\Users\Maxwe\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2026/01/03 11:11:58 INFO mlflow.tracking.fluent: Experiment with name 'model_search_guide' does not exist. Creating a new experiment.
c:\Users\Maxwe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Maxwe\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch t